#### Lab 7 
#### Rouge Metrics

In [2]:
#pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [144]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "./data.csv"
from yargy.predicates import gram, is_capitalized, dictionary, eq
from gensim import utils
import wikipedia
from yargy import Parser, rule, or_

In [147]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anar_nagmetova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anar_nagmetova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/anar_nagmetova/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
df = pd.read_csv(__PATH__,sep=";",header=0)

#### Preprocessing the title to list of tokens

In [8]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [9]:
res = {}
for title in titles:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets
print(res)

{('much', 'card', 'deck', 'well', 'shuffled'): {'much': [Synset('much.n.01'), Synset('much.a.01'), Synset('much.r.01'), Synset('much.r.02'), Synset('a_lot.r.01'), Synset('much.r.04'), Synset('much.r.05')], 'card': [Synset('card.n.01'), Synset('card.n.02'), Synset('card.n.03'), Synset('card.n.04'), Synset('wag.n.01'), Synset('poster.n.01'), Synset('calling_card.n.02'), Synset('card.n.08'), Synset('menu.n.01'), Synset('batting_order.n.01'), Synset('circuit_board.n.01'), Synset('tease.v.07'), Synset('card.v.02')], 'deck': [Synset('deck.n.01'), Synset('deck.n.02'), Synset('pack_of_cards.n.01'), Synset('deck.n.04'), Synset('deck.v.01'), Synset('deck.v.02'), Synset('deck.v.03')], 'well': [Synset('well.n.01'), Synset('well.n.02'), Synset('well.n.03'), Synset('well.n.04'), Synset('well.n.05'), Synset('well.v.01'), Synset('well.a.01'), Synset('good.s.13'), Synset('well.s.03'), Synset('well.r.01'), Synset('well.r.02'), Synset('well.r.03'), Synset('well.r.04'), Synset('well.r.05'), Synset('well.r

In [221]:
def get_hyperonyms(word_1):
    synset = wn.synsets(word_1)
    if len(synset) == 0 
        return None, None
    else:
        for synset in wn.synsets(word_1):
            for hypernym in synset.hypernyms():
            #print synset, synset.hypernym()
        return synset.hypernym()


def F1Measure (a, b):
    a = set(a)
    b = set(b)
    overlap = set()
    overlap_hyp_cnt = 0
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    rec1 = overlap_hyp_cnt/len(a)
    prec = overlap_hyp_cnt/len(b)
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*rec1*prec/(rec1+prec), overlap
    
lang = 'eng'#input()
def distance(a,b):
    a=set(a)
    b=set(b)
    f1score, overlap = F1Measure(a, b)
    a=a-overlap
    b=b-overlap
    if len(a)==0 or len(b)==0:
        return 1.0-f1score
    buff_a= {}
    buff_b= {}
    buff_a1= {}
    buff_b1= {}
    if lang=='eng':
        buff_a  = get_hyperonyms(a)
        buff_b  = get_hyperonyms(b)
        buff_a1  = get_hyperonyms(buff_a)
        buff_b1  = get_hyperonyms(buff_b)
 #   elif lang=='rus':
  #      buff_a  = extract_hypernyms_rus(a)
  #      buff_b  = extract_hypernyms_rus(b)
  #      buff_a1  = extract_hypernyms_rus(buff_a)
  #      buff_b1  = extract_hypernyms_rus(buff_b)
    
    overlap_hyp_cnt = 0
    for word in a:
        for wordb in b:
            if len(buff_a[word].intersection(buff_b[wordb]))>0:
                overlap_hyp_cnt +=1
            elif len(buff_a1[word].intersection(buff_b1[wordb]))>0:
                overlap_hyp_cnt +=0.5
    rec1_hyp = overlap_hyp_cnt/len(a)
    prec_hyp = overlap_hyp_cnt/len(b)
    f1score_hyp = 2*rec1_hyp*prec_hyp/(rec1_hyp+prec_hyp) if overlap_hyp_cnt > 0 else 0
    f1res = (2*f1score+f1score_hyp)/3
    return (1.0-f1res)
        
        
    ### Put your code here
    ### В переменной synsets помимо нормализованного заголовка хранятся синсеты для каждого токена из заголовка.
    ### у синсетов есть гиперонимы про то как из брать здесь
    ### Если у токенов заголовков есть общие гиперонимы тогда заголовки связаны несмотря на то что слова разные
    

In [ ]:
#return float((100-fuzz.ratio(a,b))/100)

buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri]=distance(ll[0],rr[0])
print(dist) 

#### Top ten closest articles with fuzzy metrics of titles

In [0]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])


interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…

In [0]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=995), Output()), _dom_classes=('widget-interac…

In [0]:
def get_hyperonyms(main_word):
    hyper1 = set()
    hyper2 = set()
    for word in main_word:
        try:
            #word = re.sub("[^a-zA-Z]", '', word)
            tokens = nltk.word_tokenize(word)
            tags = nltk.pos_tag(tokens)
            doc_synsets = []
            for token, tag in zip(tokens, tags):
                syns = wn.synsets(word)[0]
                #doc_synsets.append(syns[0])
                #sns = wn.synsets(word)[0]
                buff_h = syns.hypernyms()
                hyper1 = buff_h[0]
                hyper2 = buff_h[1]
        except IndexError:
            continue
    return hyper1, hyper2


    #HYPONYM = eq(main_word)
    #parser = Parser(rule) 
    #text = wn.synsets(wrd)[0]
    #try:
    #    text = utils.deaccent(wikipedia.summary(main_word))
    #except wikipedia.DisambiguationError as e:
    #    s = random.choice(e.options)
    #    text = utils.deaccent(wikipedia.summary(s))
    
    #print(text)
    #text = re.sub(r'\(.+?\)', '', text)
    #for idx, match in enumerate(Parser.findall(text.lower())):
    #    k = [_.value for _ in match.tokens]
        #print(k)
        
def F1Measure(a, b):
    a = set(a)
    b = set(b)
    overlap = set()
    overlap_hyp_cnt = 0
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    recall = overlap_hyp_cnt/len(a)
    precision = overlap_hyp_cnt/len(b)
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*recall*precision/(recall+precision), overlap